Show how to use pretrained assertion status

In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.pretrained import ResourceDownloader

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

In [2]:
spark = SparkSession.builder \
    .appName("assertion-status")\
    .master("local[1]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .getOrCreate()

Create some data for testing purposes

In [3]:
from pyspark.sql import Row
R = Row('sentence', 'start', 'end')
test_data = spark.createDataFrame([R('Sister with stomach cancer .',2,3),
                      R('A thallium stress test showed tachycardia and severe dyspnea',5,5),
                      R('Positive for shortness of breath, no cough',2,4),
                      R('Positive for shortness of breath, no cough',7,7)])

Create some pipelines, one for each type of assertion classification algorithm, model download can take some time.

In [4]:
import time

# instantiate the downloader
downloader = ResourceDownloader()

embeddingsFile = 'PubMed-shuffle-win-2.bin'
embeddingsUrl = 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/PubMed-shuffle-win-2.bin'
# this may take a couple minutes
if not Path(embeddingsFile).is_file():
    urlretrieve(embeddingsUrl, embeddingsFile)

documentAssembler = DocumentAssembler() \
    .setInputCol("sentence") \
    .setOutputCol("document")

# download logistic regression based assertion status trained on negex dataset
assertion_fast_lg = downloader.downloadModel(AssertionLogRegModel, "as_fast_lg", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("assertion") \

# download bidirectional lstm based assertion status trained on negex dataset
assertion_fast_dl = downloader.downloadModel(AssertionDLModel, "as_fast_dl", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("assertion") \
    
# download bidirectional lstm based assertion status trained on i2b2 dataset
assertion_full_dl = downloader.downloadModel(AssertionDLModel, "as_full_dl", "en") \
    .setInputCols(["document"])\
    .setOutputCol("assertion")
        

finisher = Finisher() \
    .setInputCols(["assertion"]) \
    .setIncludeKeys(True)

pipeline_fast_lg = Pipeline(stages = [documentAssembler, assertion_fast_lg, finisher])
pipeline_fast_dl = PipelineModel(stages = [documentAssembler, assertion_fast_dl, finisher])
pipeline_full_dl = Pipeline(stages = [documentAssembler, assertion_full_dl, finisher])

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1, localhost, executor driver): java.io.InvalidClassException: org.apache.spark.ml.PipelineStage; local class incompatible: stream classdesc serialVersionUID = 3275105016155696140, local class serialVersionUID = 8307843718209149723
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:687)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1876)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1745)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1876)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1745)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1876)
	at java.io.ObjectInputStream.readClassDesc(

Now let's use these pipelines and see the results

In [ ]:
pipeline_fast_lg.transform(test_data).show()
pipeline_fast_dl.transform(test_data).show()
pipeline_full_dl.transform(test_data).show()